In [1]:
import matplotlib.pyplot as plt
from pysheds.grid import Grid
import pyproj
import numpy as np
import cv2
from skimage.util import view_as_blocks
from pathlib import Path
import georasters as gr
from geopandas.io import file
import cv2
from skimage.morphology import skeletonize
import os

def extract_patches_from_raster():
    count = 0
    for raster_file in Path('./world_map').glob('**/*.tif'):
        data = gr.from_file(str(raster_file))
        data = data[0:3600,0:3600]
        # print(np.min(data),np.max(data),np.mean(data))
        if (data.nodata_value is None):
            data.nodata_value = 0
        
        raster_blocks = view_as_blocks(data.raster, (225, 225))
        
        for i in range(raster_blocks.shape[0]):
            for j in range(raster_blocks.shape[1]):
                raster_data = raster_blocks[i, j]

#                 src = cv2.pyrDown(raster_data,dstsize=(225,225))
                src = raster_data
                        
                print(raster_file,i,j,np.max(src))

                data_out_downsampled = gr.GeoRaster(
                    src,
                    data.geot,
                    nodata_value=data.nodata_value,
                    projection=data.projection,
                    datatype=data.datatype,
                )
                data_out_downsampled.to_tiff(
                    './data_downsampled_blurred/data_q' + str(count) +'_'+ str(i) +'_'+ str(j))

                count += 1


def compute_rivers(tiff_image):
    grid = Grid.from_raster(str(tiff_image), data_name='dem')
    dem = grid.read_raster(str(tiff_image))
    depressions = grid.detect_pits(dem)
    dep = np.array(depressions, dtype=np.uint8)

    return np.expand_dims(dep, axis=-1)


def compute_ridges(tiff_image):

    grid = Grid.from_raster(str(tiff_image), data_name='dem')
    dem = grid.read_raster(str(tiff_image))
    dem = dem.max() - dem
    peaks = grid.detect_pits(dem)
    pks = np.array(peaks, dtype=np.uint8)

    return np.expand_dims(pks, axis=-1)


def compute_sketches():
    count = 0
    height_maps = []
    sketch_maps = []
    _l = len(os.listdir('./data_downsampled_blurred'))
    for filename in Path('./data_downsampled_blurred').glob('**/*.tif'):
        count += 1
#         file_path = str(filename)
#         file_id = file_path.split('/')
#         detailed_data = gr.from_file('./data/' + file_id[-1])
        data = gr.from_file(str(filename))
#         print('data mean:',data.mean())
        if data.mean() < 5:
#             print('killed one err1')
            continue
        
        height_map = np.array(data.raster, dtype=np.float32)
        height_map = np.expand_dims(height_map, axis=-1)
#         print('max - min height:',np.amax(height_map) - np.amin(height_map))
        if np.amax(height_map) - np.amin(height_map)<10:
#             print('killed one err2')
            continue
        if np.amin(height_map)<-10000:
            print('killed one err3')
            continue
            
        peaks = compute_ridges(filename)
        basins = compute_rivers(filename)
        height_map = (height_map - np.amin(height_map)) / \
            (np.amax(height_map) - np.amin(height_map))
        height_map = height_map * 2 - 1

        sketch_map = np.stack((peaks, basins), axis=2)
        sketch_map = np.squeeze(sketch_map, axis=-1)
        
        if count%100==1:
            print('progress',count,'/',_l, sketch_map.shape, filename)
        height_maps.append(height_map)
        sketch_maps.append(sketch_map)
#         if count > 20000:
#             break

    training_output = np.array(height_maps, dtype=np.float32)
    training_input = np.array(sketch_maps, dtype=np.float32)
    np.savez('training_data4.npz', x=training_input, y=training_output)



/home/cxhpc/.conda/envs/zc_terrain/lib/python3.6/site-packages/numba/np/ufunc/parallel.py:365: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/cxhpc/.conda/envs/zc_terrain/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
extract_patches_from_raster()


world_map/53.tif 0 0 470.2601
world_map/53.tif 0 1 464.13913
world_map/53.tif 0 2 459.10846
world_map/53.tif 0 3 443.62042
world_map/53.tif 0 4 430.23743
world_map/53.tif 0 5 410.60394
world_map/53.tif 0 6 399.61963
world_map/53.tif 0 7 397.23535
world_map/53.tif 0 8 393.7125
world_map/53.tif 0 9 393.38055
world_map/53.tif 0 10 394.27695
world_map/53.tif 0 11 397.79926
world_map/53.tif 0 12 400.60605
world_map/53.tif 0 13 386.31592
world_map/53.tif 0 14 379.50598
world_map/53.tif 0 15 341.52048
world_map/53.tif 1 0 421.51328
world_map/53.tif 1 1 416.51328
world_map/53.tif 1 2 411.51328
world_map/53.tif 1 3 412.46005
world_map/53.tif 1 4 402.7705
world_map/53.tif 1 5 394.53537
world_map/53.tif 1 6 390.62518
world_map/53.tif 1 7 399.61963
world_map/53.tif 1 8 400.91617
world_map/53.tif 1 9 391.4953
world_map/53.tif 1 10 390.5195
world_map/53.tif 1 11 394.30618
world_map/53.tif 1 12 397.07312
world_map/53.tif 1 13 400.00143
world_map/53.tif 1 14 391.31067
world_map/53.tif 1 15 381.54892
w

world_map/67.tif 8 11 1545.7748
world_map/67.tif 8 12 1416.6263
world_map/67.tif 8 13 1496.8833
world_map/67.tif 8 14 1175.8351
world_map/67.tif 8 15 999.0892
world_map/67.tif 9 0 314.29562
world_map/67.tif 9 1 242.98756
world_map/67.tif 9 2 1028.079
world_map/67.tif 9 3 1356.8048
world_map/67.tif 9 4 1229.2496
world_map/67.tif 9 5 1180.799
world_map/67.tif 9 6 1055.6534
world_map/67.tif 9 7 1160.3661
world_map/67.tif 9 8 1176.9677
world_map/67.tif 9 9 1394.4017
world_map/67.tif 9 10 1420.4515
world_map/67.tif 9 11 1472.6648
world_map/67.tif 9 12 1477.9054
world_map/67.tif 9 13 1751.8383
world_map/67.tif 9 14 1430.2466
world_map/67.tif 9 15 575.5641
world_map/67.tif 10 0 319.18695
world_map/67.tif 10 1 182.96548
world_map/67.tif 10 2 1054.7516
world_map/67.tif 10 3 1108.4037
world_map/67.tif 10 4 1242.4972
world_map/67.tif 10 5 1060.6246
world_map/67.tif 10 6 1247.5061
world_map/67.tif 10 7 1255.3063
world_map/67.tif 10 8 1209.1364
world_map/67.tif 10 9 1460.2595
world_map/67.tif 10 10

world_map/48.tif 9 11 223.42368
world_map/48.tif 9 12 238.99594
world_map/48.tif 9 13 239.99306
world_map/48.tif 9 14 224.58107
world_map/48.tif 9 15 222.7579
world_map/48.tif 10 0 248.75484
world_map/48.tif 10 1 243.50783
world_map/48.tif 10 2 240.87291
world_map/48.tif 10 3 239.81706
world_map/48.tif 10 4 239.95786
world_map/48.tif 10 5 238.17767
world_map/48.tif 10 6 235.96693
world_map/48.tif 10 7 235.37656
world_map/48.tif 10 8 229.72641
world_map/48.tif 10 9 223.23166
world_map/48.tif 10 10 221.37523
world_map/48.tif 10 11 223.67319
world_map/48.tif 10 12 241.22372
world_map/48.tif 10 13 230.84453
world_map/48.tif 10 14 191.35172
world_map/48.tif 10 15 223.73622
world_map/48.tif 11 0 239.77103
world_map/48.tif 11 1 238.41638
world_map/48.tif 11 2 238.05626
world_map/48.tif 11 3 236.08504
world_map/48.tif 11 4 235.69994
world_map/48.tif 11 5 232.8719
world_map/48.tif 11 6 232.64354
world_map/48.tif 11 7 230.13277
world_map/48.tif 11 8 228.69176
world_map/48.tif 11 9 224.67299
worl

world_map/27.tif 0 0 1047.1316
world_map/27.tif 0 1 1025.263
world_map/27.tif 0 2 1059.0663
world_map/27.tif 0 3 1098.28
world_map/27.tif 0 4 1163.9789
world_map/27.tif 0 5 1170.3844
world_map/27.tif 0 6 1135.0065
world_map/27.tif 0 7 1142.7374
world_map/27.tif 0 8 1219.7246
world_map/27.tif 0 9 1227.2776
world_map/27.tif 0 10 1223.9205
world_map/27.tif 0 11 1200.881
world_map/27.tif 0 12 1161.0741
world_map/27.tif 0 13 1134.7557
world_map/27.tif 0 14 1131.1842
world_map/27.tif 0 15 1114.633
world_map/27.tif 1 0 1064.2786
world_map/27.tif 1 1 1061.123
world_map/27.tif 1 2 1081.4263
world_map/27.tif 1 3 1067.7816
world_map/27.tif 1 4 1247.072
world_map/27.tif 1 5 1251.3143
world_map/27.tif 1 6 1170.7278
world_map/27.tif 1 7 1189.3165
world_map/27.tif 1 8 1212.1339
world_map/27.tif 1 9 1224.8647
world_map/27.tif 1 10 1223.3721
world_map/27.tif 1 11 1216.9354
world_map/27.tif 1 12 1185.3934
world_map/27.tif 1 13 1147.6111
world_map/27.tif 1 14 1141.0184
world_map/27.tif 1 15 1145.8705
wor

world_map/39.tif 9 8 2598.0
world_map/39.tif 9 9 2556.0
world_map/39.tif 9 10 2037.0
world_map/39.tif 9 11 1912.0
world_map/39.tif 9 12 1682.0
world_map/39.tif 9 13 1700.0
world_map/39.tif 9 14 2165.0
world_map/39.tif 9 15 1995.0
world_map/39.tif 10 0 2327.0
world_map/39.tif 10 1 1968.0
world_map/39.tif 10 2 1928.0
world_map/39.tif 10 3 1666.0
world_map/39.tif 10 4 1539.0
world_map/39.tif 10 5 1554.0
world_map/39.tif 10 6 1480.0
world_map/39.tif 10 7 2112.0
world_map/39.tif 10 8 2862.0
world_map/39.tif 10 9 2950.0
world_map/39.tif 10 10 2310.0
world_map/39.tif 10 11 2467.0
world_map/39.tif 10 12 2437.0
world_map/39.tif 10 13 2029.0
world_map/39.tif 10 14 2035.0
world_map/39.tif 10 15 2358.0
world_map/39.tif 11 0 2358.0
world_map/39.tif 11 1 2119.0
world_map/39.tif 11 2 2173.0
world_map/39.tif 11 3 2192.6782
world_map/39.tif 11 4 2306.1045
world_map/39.tif 11 5 1901.0
world_map/39.tif 11 6 1548.0
world_map/39.tif 11 7 2287.0
world_map/39.tif 11 8 2300.0
world_map/39.tif 11 9 2150.0
worl

...

world_map/54.tif 9 6 434.00955
world_map/54.tif 9 7 403.28766
world_map/54.tif 9 8 372.131
world_map/54.tif 9 9 293.2153
world_map/54.tif 9 10 354.65234
world_map/54.tif 9 11 332.19046
world_map/54.tif 9 12 365.2729
world_map/54.tif 9 13 426.123
world_map/54.tif 9 14 475.23834
world_map/54.tif 9 15 525.034
world_map/54.tif 10 0 363.68375
world_map/54.tif 10 1 313.30652
world_map/54.tif 10 2 372.18506
world_map/54.tif 10 3 462.67273
world_map/54.tif 10 4 463.975
world_map/54.tif 10 5 444.22528
world_map/54.tif 10 6 421.8657
world_map/54.tif 10 7 421.1978
world_map/54.tif 10 8 402.0217
world_map/54.tif 10 9 309.2395
world_map/54.tif 10 10 308.45877
world_map/54.tif 10 11 363.24637
world_map/54.tif 10 12 361.29153
world_map/54.tif 10 13 406.2442
world_map/54.tif 10 14 484.29416
world_map/54.tif 10 15 542.23145
world_map/54.tif 11 0 382.28857
world_map/54.tif 11 1 313.23828
world_map/54.tif 11 2 301.27905
world_map/54.tif 11 3 441.40424
world_map/54.tif 11 4 461.6942
world_map/54.tif 11 5 

world_map/8.tif 0 0 121.145706
world_map/8.tif 0 1 132.95941
world_map/8.tif 0 2 170.00266
world_map/8.tif 0 3 162.32278
world_map/8.tif 0 4 142.84134
world_map/8.tif 0 5 132.44102
world_map/8.tif 0 6 138.6535
world_map/8.tif 0 7 128.75157
world_map/8.tif 0 8 113.82592
world_map/8.tif 0 9 109.22144
world_map/8.tif 0 10 107.16534
world_map/8.tif 0 11 107.55056
world_map/8.tif 0 12 92.022316
world_map/8.tif 0 13 71.03103
world_map/8.tif 0 14 53.71643
world_map/8.tif 0 15 62.743004
world_map/8.tif 1 0 118.48567
world_map/8.tif 1 1 148.38617
world_map/8.tif 1 2 137.26576
world_map/8.tif 1 3 136.31453
world_map/8.tif 1 4 138.11974
world_map/8.tif 1 5 136.62807
world_map/8.tif 1 6 120.46463
world_map/8.tif 1 7 136.20631
world_map/8.tif 1 8 139.9158
world_map/8.tif 1 9 126.11988
world_map/8.tif 1 10 107.221344
world_map/8.tif 1 11 88.92821
world_map/8.tif 1 12 89.6058
world_map/8.tif 1 13 80.31829
world_map/8.tif 1 14 53.510677
world_map/8.tif 1 15 55.031414
world_map/8.tif 2 0 148.09221
worl

world_map/9.tif 9 7 123.115364
world_map/9.tif 9 8 113.62883
world_map/9.tif 9 9 107.27968
world_map/9.tif 9 10 115.92597
world_map/9.tif 9 11 146.24077
world_map/9.tif 9 12 145.27383
world_map/9.tif 9 13 138.12672
world_map/9.tif 9 14 118.77093
world_map/9.tif 9 15 104.20491
world_map/9.tif 10 0 205.93134
world_map/9.tif 10 1 193.90335
world_map/9.tif 10 2 171.18248
world_map/9.tif 10 3 152.51933
world_map/9.tif 10 4 128.6895
world_map/9.tif 10 5 120.70822
world_map/9.tif 10 6 108.84693
world_map/9.tif 10 7 99.8952
world_map/9.tif 10 8 105.50645
world_map/9.tif 10 9 112.874374
world_map/9.tif 10 10 114.07185
world_map/9.tif 10 11 130.54553
world_map/9.tif 10 12 142.81775
world_map/9.tif 10 13 131.0469
world_map/9.tif 10 14 109.861626
world_map/9.tif 10 15 98.080986
world_map/9.tif 11 0 171.80034
world_map/9.tif 11 1 177.3406
world_map/9.tif 11 2 170.18327
world_map/9.tif 11 3 134.19913
world_map/9.tif 11 4 121.946625
world_map/9.tif 11 5 106.62263
world_map/9.tif 11 6 103.54532
world_

world_map/38.tif 0 0 2150.0
world_map/38.tif 0 1 1947.0
world_map/38.tif 0 2 1888.0
world_map/38.tif 0 3 2250.0
world_map/38.tif 0 4 2064.0
world_map/38.tif 0 5 2047.0
world_map/38.tif 0 6 2000.0
world_map/38.tif 0 7 1980.0
world_map/38.tif 0 8 2041.0502
world_map/38.tif 0 9 2210.0
world_map/38.tif 0 10 2230.0
world_map/38.tif 0 11 2280.0
world_map/38.tif 0 12 2232.0
world_map/38.tif 0 13 2129.68
world_map/38.tif 0 14 2125.0
world_map/38.tif 0 15 2097.0
world_map/38.tif 1 0 2249.0
world_map/38.tif 1 1 2234.0
world_map/38.tif 1 2 2330.0
world_map/38.tif 1 3 2213.0
world_map/38.tif 1 4 2220.0
world_map/38.tif 1 5 2209.0
world_map/38.tif 1 6 2210.0
world_map/38.tif 1 7 2049.0
world_map/38.tif 1 8 2208.0
world_map/38.tif 1 9 2279.0
world_map/38.tif 1 10 2259.0
world_map/38.tif 1 11 2269.0
world_map/38.tif 1 12 2215.0
world_map/38.tif 1 13 2145.0
world_map/38.tif 1 14 2106.0
world_map/38.tif 1 15 2095.0
world_map/38.tif 2 0 2031.0
world_map/38.tif 2 1 2153.0
world_map/38.tif 2 2 2277.0
worl

world_map/3.tif 9 7 165.93439
world_map/3.tif 9 8 164.84488
world_map/3.tif 9 9 161.90678
world_map/3.tif 9 10 172.03957
world_map/3.tif 9 11 159.55716
world_map/3.tif 9 12 143.06105
world_map/3.tif 9 13 160.07426
world_map/3.tif 9 14 156.4499
world_map/3.tif 9 15 140.57211
world_map/3.tif 10 0 256.62943
world_map/3.tif 10 1 250.8082
world_map/3.tif 10 2 216.06866
world_map/3.tif 10 3 219.46849
world_map/3.tif 10 4 208.2154
world_map/3.tif 10 5 192.04791
world_map/3.tif 10 6 175.98807
world_map/3.tif 10 7 180.49324
world_map/3.tif 10 8 189.60327
world_map/3.tif 10 9 188.0587
world_map/3.tif 10 10 188.25945
world_map/3.tif 10 11 178.06148
world_map/3.tif 10 12 160.76706
world_map/3.tif 10 13 138.56558
world_map/3.tif 10 14 137.34853
world_map/3.tif 10 15 139.93364
world_map/3.tif 11 0 251.25499
world_map/3.tif 11 1 254.15208
world_map/3.tif 11 2 220.16281
world_map/3.tif 11 3 220.69208
world_map/3.tif 11 4 196.9153
world_map/3.tif 11 5 185.67482
world_map/3.tif 11 6 181.28064
world_map/

In [3]:
compute_sketches()

progress 1 / 18432 (225, 225, 2) data_downsampled_blurred/data_q4754_9_2.tif
progress 101 / 18432 (225, 225, 2) data_downsampled_blurred/data_q17892_14_4.tif
progress 201 / 18432 (225, 225, 2) data_downsampled_blurred/data_q14111_1_15.tif
progress 301 / 18432 (225, 225, 2) data_downsampled_blurred/data_q5622_15_6.tif
progress 401 / 18432 (225, 225, 2) data_downsampled_blurred/data_q4043_12_11.tif
progress 501 / 18432 (225, 225, 2) data_downsampled_blurred/data_q3287_13_7.tif
progress 701 / 18432 (225, 225, 2) data_downsampled_blurred/data_q18100_11_4.tif
progress 801 / 18432 (225, 225, 2) data_downsampled_blurred/data_q16796_9_12.tif
progress 901 / 18432 (225, 225, 2) data_downsampled_blurred/data_q5510_8_6.tif
progress 1001 / 18432 (225, 225, 2) data_downsampled_blurred/data_q10737_15_1.tif
progress 1201 / 18432 (225, 225, 2) data_downsampled_blurred/data_q12247_13_7.tif
progress 1301 / 18432 (225, 225, 2) data_downsampled_blurred/data_q12173_8_13.tif
progress 1401 / 18432 (225, 225, 

progress 10501 / 18432 (225, 225, 2) data_downsampled_blurred/data_q3859_1_3.tif
progress 10601 / 18432 (225, 225, 2) data_downsampled_blurred/data_q15290_11_10.tif
progress 10701 / 18432 (225, 225, 2) data_downsampled_blurred/data_q3116_2_12.tif
progress 10801 / 18432 (225, 225, 2) data_downsampled_blurred/data_q3627_2_11.tif
progress 10901 / 18432 (225, 225, 2) data_downsampled_blurred/data_q8686_14_14.tif
progress 11001 / 18432 (225, 225, 2) data_downsampled_blurred/data_q7442_1_2.tif
progress 11101 / 18432 (225, 225, 2) data_downsampled_blurred/data_q9955_14_3.tif
progress 11201 / 18432 (225, 225, 2) data_downsampled_blurred/data_q16302_10_14.tif
progress 11301 / 18432 (225, 225, 2) data_downsampled_blurred/data_q6613_13_5.tif
progress 11401 / 18432 (225, 225, 2) data_downsampled_blurred/data_q7452_1_12.tif
progress 11501 / 18432 (225, 225, 2) data_downsampled_blurred/data_q9093_8_5.tif
progress 11601 / 18432 (225, 225, 2) data_downsampled_blurred/data_q2694_8_6.tif
progress 11701 